In [1]:
import requests,configparser
def get_cookie():
    config = configparser.ConfigParser()
    config.read('secrets.txt')
    cookie = config['session_info']['cookie']
    return cookie
def get_inputs(day):
    cookie, day = get_cookie(), int(day)
    headers = {'session': cookie}
    url = f'https://adventofcode.com/2022/day/{day}/input'
    session = requests.Session()
    resp = session.get(url,cookies=headers)
    return resp.text.split('\n')[:-1]

In [ ]:
# DAY1
import numpy as np

def max_elf_snacks(data,top):
    elf_food = [0]
    for x in data:
        if x != '':
            elf_food[-1] += int(x)
        else:
            elf_food.append(0)
    top_elves = np.argpartition(elf_food,-top)[-top:]
    return sum(np.array(elf_food)[top_elves])

print(f'Part 1: {max_elf_snacks(get_inputs(1),1)}')
print(f'Part 2: {max_elf_snacks(get_inputs(1),3)}')


In [ ]:
# DAY2
rps = {'A': [1,0,0],'B': [0,1,0], 'C': [0,0,1]}
rewards = np.array([[3,0,6], [6,3,0], [0,6,3]])

def rps_score(p,q):
    return np.matmul(np.matmul(p,rewards.T),q.T).trace() + sum(np.matmul([1,2,3],q.T))

def get_strategy(d, mode = 0):
    if mode == 0:
        # simple mode
        s = {'X': [1,0,0], 'Y': [0,1,0], 'Z': [0,0,1]}
    else:
        # part 2
        s = {'X': np.roll(rps[d[0]],-1),
            'Y': rps[d[0]],
            'Z': np.roll(rps[d[0]],+1)}
    return s[d[1]]
        
    
def get_moves(data, mode = 0):    
    p = np.array([rps[d[0]] for d in data])
    q = np.array([get_strategy(d,mode) for d in data])
    return p,q

def day2(d, mode = 0):
    data = [tuple(x.split(' ')) for x in d]
    return rps_score(*get_moves(data, mode))

print(f'Part 1: {day2(get_inputs(2),0)}')
print(f'Part 2: {day2(get_inputs(2),1)}')

In [ ]:
# DAY3
def get_priority(item):
    return ord(item) - (96 if item.islower() else 64-26) 
        
def common_item(sack):
    h = int(len(sack)/2)
    c = set(sack[:h]).intersection(set(sack[h:]))
    for i in c:    
        return get_priority(i)

def common_item_3elf(sacks):
    c = set.intersection(*[set(sack) for sack in sacks])
    for i in c:    
        return get_priority(i)

def day3_p1():
    return sum(common_item(x) for x in get_inputs(3))

def day3_p2():
    data, n = get_inputs(3), 3
    chunk3 = [data[i:i+n] for i in range(0,len(data),n)]
    return sum(common_item_3elf(x) for x in chunk3)

print(f'Part 1: {day3_p1()}')
print(f'Part 2: {day3_p2()}')

In [ ]:
# DAY4
import re
s = [[int(x) for x in re.findall('(\d+)',t)] for t in get_inputs(4)]
day4_p1 = sum(1 for x in s if (x[0]-x[2])*(x[1]-x[3])<=0)
day4_p2 = sum(1 for x in s if max(x)-min(x)<=x[3]-x[2]+x[1]-x[0])
print(f'Part 1: {day4_p1}')
print(f'Part 2: {day4_p2}')

In [ ]:
# DAY5
def get_stacks(data):
    L = [[x for i,x in enumerate(line) if i%4==1] for line in d5[:d5.index('')-1]]
    L = (np.array(L).T).tolist()
    L = [l[::-1] for l in L]
    for l in L:
        while l[-1]==' ':
            l.pop()
    return L

def get_move_instructions(data):
    return [tuple(int(t) for t in re.findall('\d+',x)) for x in data if x.startswith('move') ]

def move_containers(stack,model,count,source,dest):
    x = [stack[source-1].pop() for i in range(count)]
    if model == 9000:
        stack[dest-1] += x
    else:
        stack[dest-1] += x[::-1]

def day5(model):
    d5 = get_inputs(5)
    stacks = get_stacks(d5)
    instructions = get_move_instructions(d5)
    for i in instructions:
        move_containers(stacks, model, *i)
    return ''.join([s.pop() for s in stacks])

print(f'Part 1: {day5(9000)}')
print(f'Part 2: {day5(9001)}')

In [3]:
# DAY6
d6 = get_inputs(6)[0]
def day6(data,n):
    c, j = 0, 0
    while j < n:
        nidx = data[c+1:c+n-j].find(data[c])
        c+=1
        j = 0 if nidx >= 0 else j+1
    return c

print(f'Part 1: {day6(d6,4)}')
print(f'Part 2: {day6(d6,14)}')

Part 1: 1544
Part 2: 2145


In [2]:
# DAY7
from collections import namedtuple
import re

aFile = namedtuple("aFile","name,filesize")        
class aDir:
    def __init__(self,name,parent=None):
        self.name = name
        self.parent = parent
        self.files = []
        self.dirs = []
        if parent:
            parent.add_dir(self)
    
    def add_file(self,afile):
        self.files.append(afile)
        
    def add_dir(self,adir):
        self.dirs.append(adir)
        
    def get_size(self):
        sum_files = sum(int(f.filesize) for f in self.files)
        sum_dirs = sum(d.get_size() for d in self.dirs)
        return sum_files + sum_dirs
    
    def __repr__(self):
        return f'Dir: {self.name},{int(self.get_size())}'

def sum_dirs_under(pos, N):
    x = []
    if pos.get_size() <= N:
        if pos.dirs == []:
            return pos.get_size()
        else:
            return pos.get_size()+sum(sum_dirs_under(p,N) for p in pos.dirs)
    else:
        return sum(sum_dirs_under(p,N) for p in pos.dirs)
        
def min_dirs_over(pos, N):
    x = []
    if pos.get_size() >= N:
        if pos.dirs == []:
            return pos.get_size()
        else:
            return min([min_dirs_over(p,N) for p in pos.dirs])
    else:
        return pos.parent.get_size()

def get_root(data):
    pos = None
    for line in data:
        if line == '$ cd ..':
            pos = pos.parent
            continue
        if re.findall('\$ cd',line):
            pos = aDir(line.split(' ')[2], pos)
            continue
        if re.findall('\$ ls',line):
            continue
        if line.split(' ')[0].isnumeric():
            pos.add_file(aFile(*line.split(' ')[::-1]))

    while pos.name != '/':
        pos = pos.parent
    return pos

def day7():
    root = get_root(get_inputs(7))
    
    need_to_free = root.get_size() - 40000000

    print(f'Part 1: {sum_dirs_under(root, 100000)}')
    print(f'Part 2: {min_dirs_over(root, need_to_free)}')
    

day7()


Part 1: 1427048
Part 2: 2940614


In [486]:
# DAY8
import numpy as np
d8 = '''30373
25512
65332
33549
35390'''.split('\n')
d8 = get_inputs(8)

def transform_day8(data):
    d = [list(i) for i in data]
    d = [[int(i) for i in x] for x in d]
    return np.array(d)

def total_visibles(data):
    # convert to 2d-array
    t = transform_day8(data)
    
    # calculate directional visibility
    left_vis = np.roll(np.maximum.accumulate(t,axis=1),1,axis=1)
    top_vis = np.roll(np.maximum.accumulate(t,axis=0),1,axis=0)
    bottom_vis = np.roll(np.flip(np.maximum.accumulate(np.flip(t),axis=0)),-1,axis=0)
    right_vis = np.roll(np.flip(np.maximum.accumulate(np.flip(t),axis=1)),-1,axis=1)
    
    # minimum visibility status
    A = np.minimum.reduce([left_vis,top_vis,bottom_vis,right_vis])   
    return A.size - np.array(A>=t)[1:-1,1:-1].sum()

def get_view(r,c,m):
    # edges have view = 0
    if r in (0, m.shape[0]-1) or c in (0, m.shape[1]-1):
        return 0
    
    # return view depth, given the list of findings
    g = lambda x: x[0]+1 if len(x)>0 else None
    
    row, col  = m[r,:], m[:,c]
    # find number of visible trees in each direction
    right = g(np.where(row[c+1:]>=m[r,c])[0]) or m.shape[1]-c-1
    left = g(np.where(row[c-1::-1]>=m[r,c])[0]) or c
    down = g(np.where(col[r+1:]>=m[r,c])[0]) or m.shape[0]-r-1
    up = g(np.where(col[r-1::-1]>=m[r,c])[0]) or r
    
    return right*left*down*up

def find_best_view(data):
    # convert to 2d-array
    t = transform_day8(data)
    
    # iterate through the matrix
    max_view = 0
    for i,_ in enumerate(t):
        for j,_ in enumerate(t[i]):
            k = get_view(*(i,j),t)
            max_view = max(max_view,k)
    return max_view

print(f'Part 1: {total_visibles(d8)}')
print(f'Part 2: {find_best_view(d8)}')


Part 1: 1688
Part 2: 410400
